# Black Holes at Home Tutorial: Using Python in VisIt

## Author: Tyndale Stutzman

## This tutorial demonstrates how to plot Synthetic Data in Visit via Python

## Introduction: 

Some quick information on how python is used in conjunction with visit... little confusing 


<a id='toc'></a>
# Table of Contents:
$$\label{toc}$$
<!--these buttons aren't functioning properly so maybe figure that out-->
1. [Step 1](#installing_visit): Installing VisIt
1. [Step 2](#installing_modules): Installing necessary python Modules
1. [Step 3](#generation): Generate synthetic data
1. [Step 4](#run): Python to VisIt
1. [Step 5](#tune): Fine Tuning Visit
1. [Step 6](#conclusion): Conclusion

# Step 1: Installing VisIt \[Back to [top](#toc)\]
$$\label{installing_visit}$$
<a id='installing_visit'></a>

<!--Add some fancy buttons etc. to skip to the apropriate section by OS respectively-->
This tutorial covers prebuilt installation. Compiling from the source will be implemented in the future.  

1. [GNU/OSX](#gnu_osx): <!--comments/suggestions re this OS-->
1. [Windows](#windows): <!--comments/suggestions re this OS-->

# Step 2: Installing necessary Python modules \[Back to [top](#toc)\]
$$\label{installing_modules}$$
<a id='installing_modules'></a>

<!--Add some fancy buttons etc. to skip to the apropriate section by OS respectively-->
Instructions are provided for your given operating system below. 

1. [GNU/OSX](#gnu_osx): <!--comments/suggestions re this OS-->
1. [Windows](#windows): <!--comments/suggestions re this OS-->

### GNU/OSX \[Back to [top](#installing_modules)\] <!--maybe this is totally unecessary...?-->

In [ ]:
pip3 install scipy matplotlib math numpy visit

### Windows \[Back to [top](#installing_modules)\] <!--assuming pip3 install works on windows too, this entire cell can be deleted-->

# Step 3: Generate synthetic data \[Back to [top](#toc)\]
$$\label{generation}$$
<a id='generation'></a>

Here we will use the script from the previous notebook to generate synthetic data in the current directory

In [ ]:
from scipy import special
import matplotlib as plt
from math import pi,cos,sin

#error function
def ERF(x,x0,w):
    return 0.5 * (special.erf(((x)-(x0))/(w)) + 1.0)

destination_directory = r"./"
t_final = 2000
num_data_pts = 1000
deltat = (t_final)/num_data_pts
BH1radius = 0.25
BH2radius = 0.25
orbital_period = 225
omega = 2*pi/orbital_period

#Replace 'synthetic_data.txt' here  
with open(destination_directory + "synthetic_data_BH1Only.csv", "w") as file:
    file.write("time BH1x BH1y BH1z \n") #add headers if BH2 is included in the file
    for i in range(num_data_pts):
        time = deltat * i 
        orbital_separation = ERF(time, 1000, -500) 
        #BH1 coords
        BH1x = 0.5 * orbital_separation * cos(omega * time)
        BH1y = 0.5 * orbital_separation * sin(omega * time)
        BH1z = 0
        #BH2 coords
        BH2x = -0.5 * orbital_separation * cos(omega * time)
        BH2y = -0.5 * orbital_separation * sin(omega * time)
        BH2z = 0
        
        #typecast 
        outstr = str(time) + "," + str(BH1x) + "," + str(BH1y) + "\n" #+ str(BH2x) + " " + str(BH2y) + "\n"
        file.write(outstr)

# Step 4: Python to Visit \[Back to [top](#toc)\]
$$\label{run}$$
<a id='run'></a>

With the synthetic data in the current directory, we can use python to create a sphere with coordinates defined by the data in the CSV file

In [ ]:
import sys
sys.path.append("yourpath") # this is only necessary if the module fails to be accessed properly
import visit
import csv

def create_sphere(x, y, z, radius):
    """Create a sphere with center at (x, y, z) and given radius"""
    return x**2 + y**2 + z**2 - radius**2

def plot_sphere(x, y, z, radius):
    """Plot a sphere in VisIt using the CreateSphere method"""
    visit.OpenDatabase("sphere.silo")
    visit.DefineScalarExpression("sphere", create_sphere(x, y, z, radius))
    visit.AddPlot("Pseudocolor", "sphere")
    visit.DrawPlots()

# create x, y, z
filename = "synthetic_data_BH1xyOnly_Commas.csv"

with open(filename, 'r') as file:
    reader = csv.reader(file)
    header = next(reader)  # skip the header row
    column1, column2, column3 = [], [], []
    for row in reader:
        t = float(row[0])
        x = float(row[1])
        y = float(row[2])
''' 
or use this? 
data = np.genfromtxt('synthetic_data.txt', dtype=float, delimiter=' ', skip_header=1)
BHx1=data[:,1]
BHy1=data[:,2]
BHx2=data[:,3]
BHy2=data[:,4]
'''

z = 0
# plot
plot_sphere(x, y, z, 1)